In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=Anshul@61")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS population_data")
    cur.execute("CREATE DATABASE population_data")
    
    # close connection with default DB
    conn.close()
    
    # connect to sparkify DB
    conn = psycopg2.connect("host=127.0.0.1 dbname=population_data user=postgres password=Anshul@61")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
height_weight = pd.read_csv("height_weight_data.csv")
height_weight_clean = height_weight[['country', 'female_height', 'male_height', 'female_weight', 'male_weight']]
height_weight_clean.head()

,country,female_height,male_height,female_weight,male_weight
0,Netherlands,170,184,73.2,87.9
1,Montenegro,170,183,75.3,90.4
2,Estonia,168,182,73.7,89.9
3,Denmark,169,182,70.2,86.8
4,Bosnia and Herzegovina,167,182,70.6,87.1


In [45]:
iq_data = pd.read_csv("iq.csv")
iq_data = iq_data[iq_data['education_expenditure_per_inhabitant'].notna()]
for ind in iq_data.index:
    if "," in iq_data['education_expenditure_per_inhabitant'][ind]:
        iq_data['education_expenditure_per_inhabitant'][ind] = iq_data['education_expenditure_per_inhabitant'][ind].replace(',', "")
        # print(iq_data['education_expenditure_per_inhabitant'][ind])
iq_data.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_10244\2693341349.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iq_data['education_expenditure_per_inhabitant'][ind] = iq_data['education_expenditure_per_inhabitant'][ind].replace(',', "")


,country,iq,education_expenditure_per_inhabitant,daily_max_temp
0,Singapore,108,908,31.5
1,Hong Kong,108,915,26.2
3,South Korea,106,537,18.2
4,Japan,105,1258,19.2
5,China,104,27,19.0


In [7]:
life_expectancy = pd.read_csv("life_expectancy.csv")
life_expectancy.head()

,country,male_life_expectancy,female_life_expectancy,birth_rate,death_rate
0,Hong Kong,82.9,88.0,5.8,6.8
1,Iceland,81.7,84.5,12.3,7.8
2,Japan,81.6,87.7,6.8,8.8
3,Norway,81.6,84.9,9.8,9.8
4,Singapore,81.5,86.1,8.5,10.8


In [9]:
cur, conn = create_database()

In [10]:
height_weight_tableCreate = ("""CREATE TABLE IF NOT EXISTS heightWeight(
country VARCHAR PRIMARY KEY,
female_height INT,
male_height INT,
female_weight FLOAT(2),
male_weight FLOAT(2)
)""")

In [11]:
cur.execute(height_weight_tableCreate)
conn.commit()

In [12]:
iq_tableCreate = ("""CREATE TABLE IF NOT EXISTS iqdata(
country VARCHAR PRIMARY KEY,
iq INT,
education_expenditure_per_inhabitant INT,
daily_max_temp FLOAT(2)
)""")

In [13]:
cur.execute(iq_tableCreate)
conn.commit()

In [14]:
life_expectancy_tableCreate = ("""CREATE TABLE IF NOT EXISTS lifeExpectancy(
country VARCHAR PRIMARY KEY,
male_life_expectancy FLOAT(2),
female_life_expectancy FLOAT(2),
birth_rate FLOAT(2),
death_rate FLOAT(2)
)""")

In [15]:
cur.execute(life_expectancy_tableCreate)
conn.commit()

In [17]:
height_weight_tableInsert = ("""INSERT INTO heightweight(
country,
female_height,
male_height,
female_weight,
male_weight)
VALUES(%s, %s, %s, %s, %s)
""")

In [18]:
for i, row in height_weight_clean.iterrows():
    cur.execute(height_weight_tableInsert, list(row))

In [19]:
conn.commit()

In [26]:
iq_tableInsert = ("""INSERT INTO iqdata(
country,
iq,
education_expenditure_per_inhabitant,
daily_max_temp)
VALUES(%s, %s, %s, %s)
""")

In [44]:
conn.rollback()
try:
    for i, row in iq_data.iterrows():
        cur.execute(iq_tableInsert, list(row))
except psycopg2.Error as e:
    print("Error: Cannot insert")
    print(e)

In [46]:
conn.commit()

In [47]:
life_expectancy_tableInsert = ("""INSERT INTO lifeExpectancy(
country,
male_life_expectancy,
female_life_expectancy,
birth_rate,
death_rate)
VALUES(%s, %s, %s, %s, %s)
""")

In [48]:
try:
    for i, row in life_expectancy.iterrows():
        cur.execute(life_expectancy_tableInsert, list(row))
except psycopg2.Error as e:
    print("Error: Cannot insert")
    print(e)

In [49]:
conn.commit()